<a href="https://colab.research.google.com/github/nazihkhelifa/HW/blob/main/Recomendation_search_Phd_Title.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity , linear_kernel
from sklearn.feature_extraction.text import CountVectorizer , TfidfVectorizer


In [ ]:
pip install streamlit 

     |████████████████████████████████| 9.3 MB 4.2 MB/s 
     |████████████████████████████████| 164 kB 69.9 MB/s 
     |████████████████████████████████| 180 kB 70.5 MB/s 
     |████████████████████████████████| 4.3 MB 51.8 MB/s 
     |████████████████████████████████| 76 kB 4.7 MB/s 
     |████████████████████████████████| 111 kB 73.9 MB/s 
     |████████████████████████████████| 63 kB 1.8 MB/s 
     |████████████████████████████████| 127 kB 57.2 MB/s 
     |████████████████████████████████| 792 kB 70.8 MB/s 
     |████████████████████████████████| 374 kB 70.2 MB/s 
  Created wheel for blinker: filename=blinker-1.4-py3-none-any.whl size=13478 sha256=a8c2d49e66a57f549faa091ea5880b206979feb12f20bbb59ce6ae6ab491efe7
  Stored in directory: /root/.cache/pip/wheels/22/f5/18/df711b66eb25b21325c132757d4314db9ac5e8dabeaf196eab
Successfully built blinker
  Attempting uninstall: prompt-toolkit
    Found existing installation: prompt-toolkit 1.0.18
    Uninstalling prompt-toolkit-1.0.18:
      S

In [ ]:
import streamlit as st

In [ ]:
st.write("nazih test")

2022-01-17 15:34:29.828 
  command:

    streamlit run /usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py [ARGUMENTS]


In [ ]:
data = pd.read_excel('/content/pfe_with_details.xlsx')
data.head()

,Année,Cote,N° Exemplaire,Titre propre,Complément du titre,Résumé,Unnamed: 6,Auteur,auteur,Auteur,Auteur.1,Directuer,Dircteur,Mot clés
0,2000,PA00100,PA00100,Commande non linéaire d'un filtre actif parall...,NaN,Ce travail décrit la conception et la commande...,Automatique,Ait Hammouda,Islem,Cherouat,Hatem,Berkouk,El Madjid,Filtrage actif\nCompensation d'énergie réac...
1,2000,PA00200,PA00200,Application des techniques de commandes multiv...,NaN,Dans ce travail nous avons appliqué la command...,Automatique,Abdelmalek Allouache,Auteur,El-Fateh Kiram,Auteur,Rachid Illoul,Directeur de thèse,Réseau électrique\nÉquation de Lyapunov\nPl...
2,2000,PA00300,PA00300,Contribution à l'étude des perturbations condu...,NaN,L'évolution de la technologie et des structure...,Génie Électrique,Arroudj,Mohamed,Khelif,Bessam,R. Tahmi,Directeur de thèse,Convertisseur statique\nPerturbations condui...
3,2000,PA00400,PA00400,La commande LQG adaptative,application à la stabilisation d'un simulateur...,Dans notre travail nous avons traité la comman...,Automatique,Atek,Sofiane,Dahia,Karim,Tadjine,Mohamed,LQG adaptaive -- Commande\nLQG adaptaive ...
4,2000,PA00500,PA00500,"Commande combinée floue, mode glissant et neur...",application en robotique,Dans ce travail nous proposons trois types de ...,Automatique,Borsali,Riad Ahmed,Braci,Mohamed,Boudjema,Fares,Logique floue\nMode glissant flou\nRéseaux ...


In [ ]:
#Extract relevant columns that would influence a book's rating based on book title. 
pfe_resume = data[['Cote', 'Titre propre', 'Résumé' ]]
pfe_resume.columns = ['Cote', 'Titrepropre', 'Résumé' ]

pfe_resume.head()

,Cote,Titrepropre,Résumé
0,PA00100,Commande non linéaire d'un filtre actif parall...,Ce travail décrit la conception et la commande...
1,PA00200,Application des techniques de commandes multiv...,Dans ce travail nous avons appliqué la command...
2,PA00300,Contribution à l'étude des perturbations condu...,L'évolution de la technologie et des structure...
3,PA00400,La commande LQG adaptative,Dans notre travail nous avons traité la comman...
4,PA00500,"Commande combinée floue, mode glissant et neur...",Dans ce travail nous proposons trois types de ...


In [ ]:
# lets wrap the above code in a function
def return_sim_books(title_id, title_matrix, vectorizer, top_n = 10):
    
    # generate sim matrix
    sim_matrix = cosine_similarity(title_matrix, title_matrix)
    features = vectorizer.get_feature_names()

    top_n_idx = np.flip(np.argsort(sim_matrix[title_id,]),axis=0)[0:top_n]
    top_n_sim_values = sim_matrix[title_id, top_n_idx]
    
    # find top n with values > 0
    top_n_idx = top_n_idx[top_n_sim_values > 0]
    scores = top_n_sim_values[top_n_sim_values > 0]
    
    
    # find features from the vectorized matrix
    sim_books_idx = pfe_resume['Résumé'].iloc[top_n_idx].index
    words = []
    for book_idx in sim_books_idx:
        try:
            feature_array = np.squeeze(title_matrix[book_idx,].toarray())
        except:
            feature_array = np.squeeze(title_matrix[book_idx,])
        idx = np.where(feature_array > 0)
        words.append([" , ".join([features[i] for i in idx[0]])])
        
    # collate results
    res = pd.DataFrame({"Resume" :  pfe_resume['Résumé'].iloc[title_id],
           "sim_books": pfe_resume['Résumé'].iloc[top_n_idx].values,"words":words,
           "scores":scores}, columns = ["Cote","Titrepropre","Résumé"])

     
    # find features from the vectorized matrix
    sim_title_idx = pfe_resume['Titrepropre'].iloc[top_n_idx].index
    word = []
    for book_id in sim_title_idx:
        try:
            feature_arr = np.squeeze(title_matrix[book_id,].toarray())
        except:
            feature_arr = np.squeeze(title_matrix[book_id,])
        idx = np.where(feature_array > 0)
        words.append([" , ".join([features[i] for i in idx[0]])])
        
    # collate results
    ress = pd.DataFrame({"title" :  pfe_resume['Titrepropre'].iloc[title_id],
           "sim_books": pfe_resume['Titrepropre'].iloc[top_n_idx].values,"words":words,
           "scores":scores}, columns = ["Cote","Titrepropre","Résumé"])
           
    
    
    return res , ress

In [ ]:
tf = TfidfVectorizer(analyzer = 'word', ngram_range = (1,2), min_df = 0, stop_words = 'english')
tfidf_matrix = tf.fit_transform(pfe_resume['Résumé'])
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)
cosine_sim
titles = pfe_resume['Résumé']
indices = pd.Series(pfe_resume.index, index = pfe_resume['Résumé']) #converting all titles into a Series



#Function that gets book recommendations based on the cosine similarity score of book titles
def resume_recommendations(title, n):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key = lambda x:x[1], reverse = True)
    sim_scores = sim_scores[1:n+1]
    book_indices = [i[0] for i in sim_scores]
    return titles.iloc[book_indices]




In [ ]:
#Recommend n books for a book having index 1
pfe_index = 10
n = 20


print(pfe_resume['Résumé'][pfe_index])
resume_recommendations(pfe_resume.Résumé[pfe_index],n)




L'objectif du travail présenté dans ce mémoire est l'étude de la commande en cascade par mode de glissement de la machine asynchrone alimentée par onduleur de tension. Ainsi, nous avons synthétisé deux commandes à savoir: une commande vectorielle avec un réglage linéaire par régulateur proportionnel-intégral (PI) et un réglage non linéaire par mode de glissement associé aux systèmes à structures variables. Cette étude a débuté par une modélisation de l'ensemble convertisseur-machine asynchrone, puis une commande par orientation du flux rotorique a été introduite, suivi d'un réglage linéaire avec un régulateur PI appliqué au système à commander. Ensuite une étude théorique a été présentée sur la commande par mode de glissement en approche discrète pour les degrés relatifs d'ordre 1 et 2. Finalement un test de robustesse concernant toutes les commandes appliquées dans ce projet a été présenté. Les résultats de simulations obtenus ont montré un comportement très satisfaisant de la machine

174     L'objectif du travail présenté dans ce mémoire...
101     L'objectif du travail présenté dans ce mémoire...
3125    Ce travail traite la modélisation, le contrôle...
3451    Le travail présenté dans ce mémoire est porté ...
483     Ce travail a pour but l'analyse et la commande...
497     Ce travail à pour but l'étude et analyse d'un ...
605     Ce travail a pour but l'analyse et la commande...
606     Ce travail a pour but l'analyse et la commande...
258     Le travail présenté dans ce mémoire est consac...
260     Ce mémoire porte sur l'utilisation des réseaux...
394     Ce travail est consacré à l'étude et la simula...
866     Ce mémoire de thèse est une contribution à la ...
867     Ce mémoire de thèse est une contribution à la ...
1150    Ce projet de fin détudes concerne lélaborati...
1151    Ce projet de fin détudes concerne lélaborati...
3446    Ce travail consiste à la modélisation et la co...
3613    Le travail présenté dans ce mémoire consiste e...
100     Ce mém

In [ ]:
t = TfidfVectorizer(analyzer = 'word', ngram_range = (1,2), min_df = 0, stop_words = 'english')
tfidf_matri = t.fit_transform(pfe_resume['Titrepropre'])
cosine_si = cosine_similarity(tfidf_matri, tfidf_matri)
cosine_si
title = pfe_resume['Titrepropre']
indice = pd.Series(pfe_resume.index, index = pfe_resume['Titrepropre']) #converting all titles into a Series


In [ ]:
def title_recommendations(title, n):
    id = indice[title]
    sim_score = list(enumerate(cosine_si[id]))
    sim_score = sorted(sim_score, key = lambda x:x[1], reverse = True)
    sim_score = sim_score[1:n+1]
    book_indice = [i[0] for i in sim_score]
    return str(title.iloc[book_indice])

In [ ]:
#print(pfe_resume['Titrepropre'][pfe_index])
title_recommendations(pfe_resume.Titrepropre[pfe_index],n)

AttributeError: ignored

In [ ]:
pfe_resume.iloc[0].Titrepropre    


"Commande non linéaire d'un filtre actif parallèle à trois niveaux"